In [1]:
from kfp.components import create_component_from_func

import sys
sys.path.insert(0, "..")
from constants import NAMESPACE, HOST
from utils.auth import get_session_cookie

In [2]:
EXPERIMENT_NAME = "tutorial"

In [3]:
def add(a: float, b: float) -> float:
    return a + b

add_op = create_component_from_func(
    func=add,
    base_image="python:3.7",
    output_component_file="../components/add/component.yaml",
    packages_to_install=["pandas==0.24"],
)

def minus(a: float, b: float) -> float:
    return a - b

minus_op = create_component_from_func(
    func=minus,
    base_image="python:3.7",
    output_component_file="../components/minus/component.yaml",
    packages_to_install=["pandas==0.24"]
)

In [4]:
from kfp import dsl
import kfp

In [5]:
@dsl.pipeline(
    name="Condition",
    description="Run minus of if the condition is satisfied."
)

def add_and_minus():
    add_task = add_op(1, 2)
    sum_output_ref = add_task.outputs["Output"]
    
    with dsl.Condition(sum_output_ref > 3):
        minus_task = minus_op(sum_output_ref, 4)
        

In [6]:
session_cookie = get_session_cookie()

client = kfp.Client(
    host = f"{HOST}/pipline/?ns=kubeflow-user#/pipelines",
    cookies=f"authservice_session={session_cookie}",
    namespace=NAMESPACE,
)

In [7]:
client.create_run_from_pipeline_func(
    add_and_minus,
    arguments={},
    experiment_name=EXPERIMENT_NAME
)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.17.10', 'Date': 'Thu, 30 Jun 2022 09:37:40 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '172', 'Connection': 'keep-alive', 'x-powered-by': 'Express', 'content-security-policy': "default-src 'none'", 'x-content-type-options': 'nosniff', 'x-envoy-upstream-service-time': '2'})
HTTP response body: <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Cannot POST /pipline/apis/v1beta1/experiments</pre>
</body>
</html>

